# helpers

> This module contains all the helper functions for this library.

In [ ]:
#| default_exp helpers

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#|eval: false
from datasets import load_dataset
from tqdm.auto import tqdm

ds = load_dataset("bigcode/the-stack-smol", data_dir="data/python", split="train")

repo_files = {}
for repo_name in tqdm(set(ds["repository_name"]), desc="Processing repos"):
    rows_w_repo = ds.filter(lambda example: example["repository_name"] == repo_name)

    if len(rows_w_repo) > 1:
        repo_files[repo_name] = [row["content"] for row in rows_w_repo]
        if len(repo_files) > 10:
            break

repo_files.keys()

Using custom data configuration bigcode--the-stack-smol-7b51f8bde3058781
Found cached dataset json (/home/nathan/.cache/huggingface/datasets/bigcode___json/bigcode--the-stack-smol-7b51f8bde3058781/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


Processing repos:   0%|          | 0/9457 [00:00<?, ?it/s]

Loading cached processed dataset at /home/nathan/.cache/huggingface/datasets/bigcode___json/bigcode--the-stack-smol-7b51f8bde3058781/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-68bcc181409b7e0b.arrow
Loading cached processed dataset at /home/nathan/.cache/huggingface/datasets/bigcode___json/bigcode--the-stack-smol-7b51f8bde3058781/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-7f7324f50a250d6f.arrow
Loading cached processed dataset at /home/nathan/.cache/huggingface/datasets/bigcode___json/bigcode--the-stack-smol-7b51f8bde3058781/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-260ea8bff837beea.arrow
Loading cached processed dataset at /home/nathan/.cache/huggingface/datasets/bigcode___json/bigcode--the-stack-smol-7b51f8bde3058781/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab/cache-9f86eaa2cac1bae9.arrow
Loading cached processed dataset at /home/nathan/.cache/huggingface/

dict_keys(['yanjingke/yolov4-keras', '0xFEEDC0DE64/homeassistant-core', 'uve/tensorflow', 'JonathanGailliez/azure-sdk-for-python', 'HaochengLIU/spack', '0gajun/integrations-core', 'HackReborn/rqalpha', 'DevRGT/home-assistant', 'Abdelrhman-Hosny/metrics', '06needhamt/intellij-community', 'Maurya232Abhishek/Python-repository-for-basics'])

In [ ]:
#| export
def get_query(language, program_lang):
    "Get a query based on the language"
    if program_lang == "python":
        return language.query("""
            (function_definition
                name: (identifier) @func.name)

            (class_definition
                name: (identifier) @class.name)
            """
        )

In [ ]:
#| export
def get_internal_methods(file_contents, tokenizer):
    """
    Get all the internal methods in a set of files
    """
    project_content = "\n\n".join(file_contents)
    tree = tokenizer.parser.parse(project_content.encode())
    root_node = tree.root_node
    query = get_query(tokenizer.language, tokenizer.program_lang)
    captures = query.captures(root_node)
    # make sure to ignore dunders
    internal_methods = {node.text.decode() for node, _ in captures if not node.text.decode().startswith("__")}
    return internal_methods

In [ ]:
#|eval: false
from code_tokenizers.core import CodeTokenizer

py_tokenizer = CodeTokenizer.from_pretrained("gpt2", "python")

internal_methods = get_internal_methods(repo_files["yanjingke/yolov4-keras"], py_tokenizer)
internal_methods

{'DarknetConv2D',
 'DarknetConv2D_BN_Leaky',
 'YOLO',
 '_get_anchors',
 '_get_class',
 'close_session',
 'detect_image',
 'generate',
 'get_defaults',
 'make_five_convs',
 'yolo_body',
 'yolo_boxes_and_scores',
 'yolo_correct_boxes',
 'yolo_eval',
 'yolo_head'}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()